In [63]:
import os
from dotenv import load_dotenv
from pathlib import Path
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import Document
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
load_dotenv()

True

In [64]:
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

In [65]:
# Load and Prepare Your Text Data

documents = []
data_folder = Path("data")

for filename in os.listdir(data_folder):
    if filename.endswith(".txt"):
        with open(os.path.join(data_folder, filename), 'r') as file:
            content = file.read()
            doc = Document(page_content=content, metadata={"source": filename})
            documents.append(doc)

In [66]:
documents

[Document(metadata={'source': 'food.txt'}, page_content='Here are the names of the food items of our restaurant:\nPepperoni Pizza ($12.99)\nDescription: A classic Italian pizza topped with tomato sauce, mozzarella cheese, and generous slices of pepperoni.\nIngredients: Tomato sauce, mozzarella cheese, pepperoni, olive oil, basil.\nClassic Burger ($8.49)\nDescription: Juicy Potato patty served with fresh lettuce, tomato, onions, and pickles in a soft bun.\nIngredients: Potato patty, lettuce, tomato, onion, pickles, burger bun, cheese (optional).\nDosa ($5.99)\nDescription: A thin, crispy South Indian crepe made from rice and lentils, served with coconut chutney and sambar.\nIngredients: Rice, urad dal (black gram), fenugreek, oil.\nPav Bhaji ($6.49)\nDescription: A spicy and flavorful mashed vegetable curry served with buttered bread rolls. A popular Indian street food.\nIngredients: Mixed vegetables, tomatoes, onions, butter, spices, pav bread.\nVada Pav ($3.99)\nDescription: A spicy p

In [67]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = text_splitter.split_documents(documents)        

In [68]:
splits

[Document(metadata={'source': 'food.txt'}, page_content='Here are the names of the food items of our restaurant:\nPepperoni Pizza ($12.99)\nDescription: A classic Italian pizza topped with tomato sauce, mozzarella cheese, and generous slices of pepperoni.\nIngredients: Tomato sauce, mozzarella cheese, pepperoni, olive oil, basil.\nClassic Burger ($8.49)'),
 Document(metadata={'source': 'food.txt'}, page_content='Classic Burger ($8.49)\nDescription: Juicy Potato patty served with fresh lettuce, tomato, onions, and pickles in a soft bun.\nIngredients: Potato patty, lettuce, tomato, onion, pickles, burger bun, cheese (optional).\nDosa ($5.99)'),
 Document(metadata={'source': 'food.txt'}, page_content='Dosa ($5.99)\nDescription: A thin, crispy South Indian crepe made from rice and lentils, served with coconut chutney and sambar.\nIngredients: Rice, urad dal (black gram), fenugreek, oil.\nPav Bhaji ($6.49)'),
 Document(metadata={'source': 'food.txt'}, page_content='Pav Bhaji ($6.49)\nDescri

In [69]:
len(splits)

10

In [70]:
# ChromaDB 

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectors = Chroma.from_documents(documents=splits, embedding=embedding, persist_directory="./chorma")

In [71]:
# Retriver

retriever = vectors.as_retriever(kwargs=3)

In [72]:
llm = ChatGroq(model_name="Gemma2-9b-It")

In [73]:
contextualize_s_prompt = ("Imagine you are a skilled conversation partner. Given the ongoing chat history and the most recent user question, your task is to craft a clear, standalone question that can be understood without needing any prior context. Please do not provide an answer; simply reformulate the question if necessary, or return it as is, ensuring it remains concise and relevant.")

In [74]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_s_prompt),
        MessagesPlaceholder("chat_hist"),
        ("human", "{input}"),
    ]
)

In [75]:
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

In [76]:
s_prompt = ("""
You are a food ordering assistant. Help users with their orders by understanding their requests and responding clearly. Focus on the following tasks:
Greet users and offer assistance.
Help them add, remove, or complete their orders.
Provide order status updates.
Address menu inquiries and item availability.
{context}
""")

In [77]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", s_prompt),
        MessagesPlaceholder("chat_hist"), 
        ("human", "{input}"),   
    ]
)

In [78]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [79]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [95]:
session_store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory() 

    return session_store[session_id]

In [96]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_hist",
    output_messages_key="answer"
)

In [99]:
def ask_question(question: str, session_id: str = "default"):
    response = conversational_rag_chain.invoke(
        {
            "input": question
        },
        config = {
            "configurable": {"session_id": session_id}
        },
    )
    
    return response

In [101]:
ask_question("Hi")

{'input': 'Hi',
 'chat_hist': [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! Welcome to our food ordering service. What can I get for you today? 😊  \n\n', additional_kwargs={}, response_metadata={})],
 'context': [Document(metadata={'source': 'food.txt'}, page_content='Pav Bhaji ($6.49)\nDescription: A spicy and flavorful mashed vegetable curry served with buttered bread rolls. A popular Indian street food.\nIngredients: Mixed vegetables, tomatoes, onions, butter, spices, pav bread.\nVada Pav ($3.99)'),
  Document(metadata={'source': 'food.txt'}, page_content='Dosa ($5.99)\nDescription: A thin, crispy South Indian crepe made from rice and lentils, served with coconut chutney and sambar.\nIngredients: Rice, urad dal (black gram), fenugreek, oil.\nPav Bhaji ($6.49)'),
  Document(metadata={'source': 'food.txt'}, page_content='Vada Pav ($3.99)\nDescription: A spicy potato fritter sandwiched in a bread roll with chutneys. Known as the In